In [ ]:
from scapy.all import * 
from scapy.all import ICMP, IP
import time
import statistics

Creamos una función ping que envía echo request al host pasado por parámetro. La función devuelve un diccionario que contiene el host contactado (dst) y:
* Tipo de respuesta ICMP recibida (response_type)
* RTT en milisegundos (rtt_ms)
* Longitud del paquete recibido (len)
* Time to live del paquete recibido (ttl) 

El valor para estas claves es None si no se obtuvo respuesta.

In [4]:
def ping(host): 

    # Diccionario de códigos ICMP para respuestas de tipo 3 (Destino inalcanzable)
    codigos = {
        0: "Red de destino inalcanzable",
        1: "Host de destino inalcanzable",
        2: "Protocolo inalcanzable",
        3: "Puerto inalcanzable",
        4: "Fragmentación requerida y DF activado",
        5: "Ruta de origen fallida"}
    
    # Creo diccionario con valores por defecto
    data = {
        "dst": host,
        "response_type": None,
        "rtt_ms": None,
        "len": None,
        "ttl": None
    }
    
    print(f"Pinging {host}...")

    packet = IP(dst=host)/ICMP(type=8) # Echo Request
    
    inicio = time.time()
    response = sr1(packet, timeout=10, verbose=False) # sr1: envío y espero una sola respuesta
    fin = time.time()
    
    if response:

        icmp_layer = response.getlayer(ICMP)

        code = icmp_layer.code

        rtt_ms = (fin - inicio) * 1000 # RTT en milisegundos
        data["rtt_ms"] = rtt_ms
        data["len"] = response.len
        data["ttl"] = response.ttl
        data["response_type"] = response.type

        if response.type == 3: 
            print(f"Destino {host} inalcanzable: " + codigos[code])
        
        elif response.type == 0: # Si la respuesta es echo reply
            print(f"Echo-reply recibida de {host}")

        else:
            print(f"Respuesta ICMP recibida de tipo {response.type}, código {icmp_layer.code}")

    else:
        print(f"No se recibió respuesta de {host}")

    return data

In [ ]:
def ping_multiple(host, cantidad_paquetes):

    res = dict()
    sent_packets = cantidad_paquetes
    received_packets = 0
    lost_packets = 0
    total_rtt = 0
    min_rtt = float('inf')
    max_rtt = float('-inf')
    lista_rtts = []
    
    for i in range(cantidad_paquetes):

        resultado = ping(host)

        if resultado["response_type"] == 0: 
            received_packets += 1
            total_rtt += resultado["rtt_ms"]
            min_rtt = min(min_rtt, resultado["rtt_ms"])
            max_rtt = max(max_rtt, resultado["rtt_ms"])
            lista_rtts.append(resultado["rtt_ms"])

        else:
            lost_packets += 1 # Si el destino es inalcanzable o no se recibe respuesta, el ping se considera perdido

    res["host"] = host
    res["Paquetes enviados"] = sent_packets
    res["Paquetes recibidos"] = received_packets
    res["Paquetes perdidos"] = lost_packets
    res["Porcentaje de paquetes perdidos"] = (lost_packets / sent_packets) 
    res["RTT total"] = total_rtt if received_packets > 0 else None
    res["RTT mínimo"] = min_rtt if min_rtt != float('inf') else None
    res["RTT máximo"] = max_rtt if max_rtt != float('-inf') else None
    res["RTT promedio"] = total_rtt / received_packets if received_packets > 0 else None
    res["Desvió estándar del RTT promedio"] = statistics.stdev(lista_rtts) if len(lista_rtts) > 1 else None
    
    return res

In [ ]:
ping('utdt.edu.ar')

Contactamos a 5 universidades de continentes distintos. También contactamos a UTDT para ver las diferencias.

In [ ]:
ping_multiple('utdt.edu.ar', 5)

In [ ]:
ping_multiple('mit.edu', 5)

In [ ]:
ping_multiple('ie.edu', 5)

In [ ]:
ping_multiple('sydney.edu.au', 5)

In [ ]:
ping_multiple('cuhk.edu.cn', 5)

In [ ]:
ping_multiple('uniben.edu', 5)